<a href="https://colab.research.google.com/github/noname1711/cryptography_theory/blob/main/hill_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from google.colab import files

In [2]:
# Chuyển ký tự thành số
def char_to_num(char):
    return ord(char) - ord('A')

In [3]:
# Chuyển số thành ký tự
def num_to_char(num):
    return chr(num + ord('A'))

In [4]:
def hill_encrypt(plaintext, key_matrix):
    plaintext = plaintext.upper()
    n = len(key_matrix)
    while len(plaintext) % n != 0:
        plaintext += 'X'
    plaintext_numbers = [char_to_num(c) for c in plaintext]
    encrypted = ""
    for i in range(0, len(plaintext_numbers), n):
        chunk = plaintext_numbers[i:i + n]
        result = np.dot(key_matrix, chunk) % 26
        encrypted += ''.join(num_to_char(num) for num in result)
    return encrypted

In [5]:
def hill_decrypt(ciphertext, key_matrix):
    n = len(key_matrix)
    det = int(np.round(np.linalg.det(key_matrix))) % 26
    det_inv = pow(det, -1, 26)
    key_inverse = (det_inv * np.round(det * np.linalg.inv(key_matrix)).astype(int) % 26) % 26
    ciphertext_numbers = [char_to_num(c) for c in ciphertext]
    decrypted = ""
    for i in range(0, len(ciphertext_numbers), n):
        chunk = ciphertext_numbers[i:i + n]
        result = np.dot(key_inverse, chunk) % 26
        decrypted += ''.join(num_to_char(num) for num in result)
    return decrypted

In [6]:
def process_file(input_file, key, mode):
    plaintext = input_file.decode("utf-8").strip()
    n = int(len(key) ** 0.5)
    key_matrix = np.array([char_to_num(c) for c in key.upper()]).reshape(n, n)
    if mode == "1":  # Encrypt
        ciphertext = hill_encrypt(plaintext, key_matrix)
        output_file_name = "output_encrypted.txt"
    else:  # Decrypt
        ciphertext = hill_decrypt(plaintext, key_matrix)
        output_file_name = "output_decrypted.txt"
    with open(output_file_name, 'w') as file:
        file.write(ciphertext)
    return output_file_name

In [7]:
mode = input("Chọn chế độ (1: Mã hóa, 2: Giải mã): ")
key = input("Nhập khóa (chuỗi ký tự, độ dài là bình phương như 4, 9): ")

if mode in ["1", "2"]:
    uploaded = files.upload()
    for fn in uploaded.keys():
        processed_file = process_file(uploaded[fn], key, mode)
    files.download(processed_file)
    print("Xử lý hoàn tất. Tải file về.")
else:
    print("Lựa chọn không hợp lệ.")

Chọn chế độ (1: Mã hóa, 2: Giải mã): 1
Nhập khóa (chuỗi ký tự, độ dài là bình phương như 4, 9): Hung


Saving test.txt to test.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Xử lý hoàn tất. Tải file về.
